<a href="https://colab.research.google.com/github/RobsonVieiraSilv/LZ77/blob/main/LZ77.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Implementação do algoritmo Lempel-Ziv 77 (LZ77)**

In [1]:
import numpy as np

def lz77_compress(data, window_size):
    compressed_data = []
    i = 0

    while i < len(data):
        match_length = 0
        match_distance = 0
        caractere = data[i]

        # Define a janela de busca
        start_window = max(0, i - window_size)
        search_buffer = data[start_window:i]

        # Procura correspondência na janela de busca
        for j in range(len(search_buffer)):
            length = 0
            while (length < len(data) - i and
                   search_buffer[j:j+length+1] == data[i:i+length+1]):
                length += 1

            # Atualiza a correspondência mais longa encontrada
            if length > match_length:
                match_length = length
                match_distance = len(search_buffer) - j

        # Adiciona o caractere atual diretamente se não houver correspondência
        if match_length == 0:
            compressed_data.append((0, 0, caractere))
            i += 1
        else:
            # Combina correspondências consecutivas, se possível
            if compressed_data and compressed_data[-1][0] == 1 and compressed_data[-1][1] == match_distance:
                compressed_data[-1] = (1, match_distance, compressed_data[-1][2] + match_length)
            else:
                compressed_data.append((1, match_distance, match_length))
            i += match_length

    return compressed_data

def lz77_decompress(compressed_data):
    decompressed_data = []

    for flag, offset, length in compressed_data:
        if flag == 0:
            decompressed_data.append(length)  # length é agora o caractere original
        else:
            start = len(decompressed_data) - offset
            for i in range(length):
                decompressed_data.append(decompressed_data[start + i])

    return ''.join(decompressed_data)

Primeiro teste: verificando se a string é convertida em uma sequência de tuplas idêntica a da sequência apresenta no livro

In [33]:
string = "ABBABBABBBAABABA"
# string = "ABABAABAAABBABAB"
# string = "1011010100010"

seq_tupla_0 = lz77_compress(string, 4) ## 4 é o comprimento da janela, ou seja, window_size = 4
print(seq_tupla_0)

[(0, 0, 'A'), (0, 0, 'B'), (1, 1, 1), (1, 3, 6), (1, 4, 2), (1, 1, 1), (1, 3, 2), (1, 2, 2)]


Recuperação da string a partir da tupla gerada:

In [34]:
rec_string = lz77_decompress(seq_tupla_0)
print(rec_string)

ABBABBABBBAABABA


Segundo teste: utilizar uma frase

In [35]:
texto = "O_rato_roeu_a_roupa_do_rei_de_Roma"
seq_tupla_1 = lz77_compress(texto, 12)
seq_tupla_1

[(0, 0, 'O'),
 (0, 0, '_'),
 (0, 0, 'r'),
 (0, 0, 'a'),
 (0, 0, 't'),
 (0, 0, 'o'),
 (1, 5, 2),
 (1, 3, 1),
 (0, 0, 'e'),
 (0, 0, 'u'),
 (1, 10, 1),
 (1, 9, 1),
 (1, 7, 3),
 (1, 6, 1),
 (0, 0, 'p'),
 (1, 6, 2),
 (0, 0, 'd'),
 (1, 6, 1),
 (1, 9, 2),
 (0, 0, 'e'),
 (0, 0, 'i'),
 (1, 7, 2),
 (1, 4, 1),
 (1, 10, 1),
 (0, 0, 'R'),
 (1, 10, 1),
 (0, 0, 'm'),
 (0, 0, 'a')]

In [36]:
rec_texto = lz77_decompress(seq_tupla_1)
rec_texto

'O_rato_roeu_a_roupa_do_rei_de_Roma'

**Codificação e decodificação com LZ77**

In [37]:
import math

def to_binary(n, length):
    """Converte um número em uma string binária de comprimento específico."""
    return bin(n)[2:].zfill(length)

def lz77_basic_encode(input_string):
    # Inicializa a lista de substrings (o dicionário de substrings) com a sequência vazia
    substrings = ['']
    encoded_output = []

    # Inicializa a posição atual na string de entrada
    i = 0
    while i < len(input_string):
        # Encontra a maior substring que já apareceu
        j = 1
        while input_string[i:i + j] in substrings and (i + j) <= len(input_string):
            j += 1
        j -= 1

        # Determina o índice (ponteiro) da substring encontrada
        substring = input_string[i:i + j]
        pointer = substrings.index(substring)

        # Converte o ponteiro para binário com o número necessário de bits
        pointer_binary = to_binary(pointer, math.ceil(math.log2(len(substrings))))

        # Define o próximo bit que não está na substring para ser o bit extra
        next_bit = input_string[i + j] if (i + j) < len(input_string) else ''

        # Adiciona o par (ponteiro em binário, next_bit) ao output codificado
        encoded_output.append((pointer_binary, next_bit))

        # Adiciona a nova substring (substring + next_bit) ao dicionário de substrings
        substrings.append(substring + next_bit)

        # Move para a próxima posição após a nova substring
        i += j + 1

    return encoded_output

def generate_encoded_string(encoded_output):
    encoded_string = []
    for i in range(len(encoded_output)):
      if i==0:
        encoded_string.append(encoded_output[i][1])
      else:
        encoded_string.append(encoded_output[i][0])
        encoded_string.append(encoded_output[i][1])
    return ''.join(encoded_string)

In [38]:
def decode_encoded_string(encoded_string):
    """Decodifica a encoded_string para obter encoded_output e input_string."""
    substrings = ['']
    encoded_output = []
    input_string = []
    i = 0

    while i < len(encoded_string):
        # Determinar o número de bits do ponteiro (baseado no tamanho atual do dicionário)
        pointer_length = math.ceil(math.log2(len(substrings)))

        if pointer_length == 0:  # Caso inicial
            pointer = 0
        else:
            # Extrair o ponteiro em binário
            pointer_binary = encoded_string[i:i + pointer_length]
            pointer = int(pointer_binary, 2)
            i += pointer_length

        # Extrair o próximo bit (se existir)
        next_bit = encoded_string[i] if i < len(encoded_string) else ''
        i += 1

        # Adicionar ao encoded_output
        encoded_output.append((to_binary(pointer, pointer_length), next_bit))

        # Reconstruir o input_string
        substring = substrings[pointer] + next_bit
        input_string.append(substring)

        # Atualizar o dicionário de substrings
        substrings.append(substring)

    # Concatenar o input_string para formar a string original
    input_string = ''.join(input_string)
    return encoded_output, input_string

Exemplo 1:

In [49]:
mensagem = "1011010100010"
encoded_tupla = lz77_basic_encode(mensagem)
encoded_msg = generate_encoded_string(encoded_tupla)
print("Sequência de tuplas:", encoded_tupla)
print("Codificação binária:", encoded_msg)

Sequência de tuplas: [('0', '1'), ('0', '0'), ('01', '1'), ('10', '1'), ('100', '0'), ('010', '0'), ('001', '0')]
Codificação binária: 100011101100001000010


In [47]:
seq_tupla_2, rec_mensagem = decode_encoded_string(encoded_msg)
print("Tuplas:",seq_tupla_2)
print("Mensagem recuperada:", rec_mensagem)

Tuplas: [('0', '1'), ('0', '0'), ('01', '1'), ('10', '1'), ('100', '0'), ('010', '0'), ('001', '0')]
Memsagem recuperada: 1011010100010


In [50]:
if rec_mensagem == mensagem:
  print(True)
else:
  print(False)

True


Exercício 7.6 do livro de David MacKay:
Encode the string **000000000000100000000000** using the basic Lempel-Ziv algorithm described above.

In [48]:
string_7_6 = "000000000000100000000000"
enconded_7_6 = lz77_basic_encode(string_7_6)
print("Tuplas:", enconded_7_6)
print("Codificação binária:", generate_encoded_string(enconded_7_6))

Tuplas: [('0', '0'), ('1', '0'), ('10', '0'), ('11', '0'), ('010', '1'), ('100', '0'), ('110', '0')]
Codificação binária: 010100110010110001100


Exercício 7.7 do livro de David MacKay: Decode the string 00101011101100100100011010101000011 that was encoded using the basic Lempel-Ziv algorithm.

In [14]:
decode_7_7 = "00101011101100100100011010101000011"

string_7_7 = "0100001000100010101000001"

In [22]:
seq_tupla, mensagem_original = decode_encoded_string(decode_7_7)
print(seq_tupla)
print(mensagem_original)

[('0', '0'), ('0', '1'), ('01', '0'), ('11', '1'), ('011', '0'), ('010', '0'), ('100', '0'), ('110', '1'), ('0101', '0'), ('0001', '1')]
0100001000100010101000001


In [45]:
if mensagem_original == string_7_7:
  print(True)
else:
  print(False)

True
